## Database thoughts

Here's what we want to be able to easily get: <br>
(1) List of players competing in game <br>
(2) List of games a player has played in <br>
(3) List of games based on date range <br>
(4) All stats for each game <br>
(5) Summation of stats for a particular player's games over a particular date range <br>

Tables <br>
(1) GAME_OVERALL<br>
(2) SKATER_GAME<br>
(3) GOALIE_GAME<br>
(4) PLAYER<br>
(5) TEAM

## Database building functions

In [96]:
import sys
import pandas as pd
import sqlalchemy
import glob
from IPython.core import display as ICD

PLAYERGAME_DIR = "./Games/"
GAMESFILE = "./Games/2015Games.csv"
PLAYER_STATS_FILE = "PlayerStats"
HOME = "Home"
AWAY = "Away"

DATABASE_LOC = "test3.db"
TABLE_GAMEOVERALL = "GAME_STATS"
TABLE_PLAYERGAME = "SKATER_GAME"
TABLE_GOALIEGAME = "GOALIE_GAME"
TABLE_PLAYER = "PLAYER"
TABLE_TEAM = "TEAM"

def get_playerstats_files(gamename):
    team1 = PLAYERGAME_DIR + gamename + '_' + PLAYER_STATS_FILE + HOME + '.csv'
    team2 = PLAYERGAME_DIR + gamename + '_' + PLAYER_STATS_FILE + AWAY + '.csv'
    return team1, team2

def gamefile_to_gamename(gamefile):
    gamename = str(gamefile)
    gamename = gamename.replace('.csv', '')
    gamename = gamename.replace("./Games\\", '')
    return gamename

def append_columns(game, team1, team2):
    d_game['GameName'] = gamename
    d_team1['GameName'] = gamename
    d_team2['GameName'] = gamename
    d_team1['Result'] = TEAM1
    
def get_gamename(game):
    return str(2015) + "_" + str(game[0])

def build_database(databasename):    
    engine = sqlalchemy.create_engine('sqlite:///' + databasename)
    d_games = pd.read_csv(GAMESFILE)
    gamenames = []
    dates = []
    for game in d_games.iterrows():
        gamename = get_gamename(game)
        gamenames.append(gamename)
        dt = datetime.datetime.strptime(game[1][0], '%Y-%m-%d')
        date = int(dt.timestamp())
        dates.append(date)
    d_games['DateTimestamp'] = dates
    d_games['GameName'] = gamenames
    d_games = d_games.drop('Notes', 1)
    d_games.OT = d_games.OT=="OT"
    d_games.to_sql(TABLE_GAMEOVERALL, engine, if_exists='replace')
    
build_database(DATABASE_LOC)

## Database lookup functions

In [ ]:
def games_in_daterange(date1, date2):
    date1 = int(date1.timestamp())
    date2 = int(date2.timestamp())
    engine = sqlalchemy.create_engine('sqlite:///' + DATABASE_LOC)
    sql_query = 'SELECT * from ' + TABLE_GAMEOVERALL + ' WHERE DateTimestamp >= ' + str(date1) + ' AND ' + 'DateTimestamp <= ' + str(date2)
    games = pd.read_sql_query(sql_query, engine)
    return games